In [4]:
import flowshape as fs
import igl
import numpy as np
import meshplot as mp
import os
from src.utilities.fin_shape_utils import plot_mesh
from src.utilities.fin_class_def import FinData
from src.utilities.functions import path_leaf
import glob2 as glob
import pandas as pd

### Load fin data

In [5]:
# get list of refined fin mesh objects
# root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/pecfin_dynamics/"
fin_mesh_list = sorted(glob.glob(os.path.join(root, "point_cloud_data", "processed_fin_data", "*smoothed_fin_mesh*")))

# load metadata
fin_stats_df = pd.read_csv(os.path.join(root, "metadata", "fin_stats.csv"))
fin_stats_df["experiment_date"] = fin_stats_df["experiment_date"].astype(str)
fin_stats_df.head()

# set write directory
figure_dir = os.path.join(root, "point_cloud_data", "fin_shape_analyses")
if not os.path.isdir(figure_dir):
    os.makedirs(figure_dir) 

### Get fin mesh

In [6]:
import plotly.colors as pc
import matplotlib.colors as mc

# Get Plotly's default discrete colors (hex format)
plotly_default_colors_hex = pc.qualitative.Plotly

# Convert hex colors to RGB triplets
plotly_rgb = [mc.hex2color(hex_color) for hex_color in plotly_default_colors_hex]

set2_colors = pc.qualitative.Set2

# Convert hex colors to RGB triplets (0-255 range)
set2_rgb = [
    list(map(int, color[color.find("(")+1 : color.find(")")].split(",")))
    for color in set2_colors
]

In [10]:
import trimesh
import flowshape as fs

data_ind = 43

flow_dir = os.path.join(figure_dir, "flow_frames")
os.makedirs(flow_dir, exist_ok=True)

# get key metadata
date_string = fin_stats_df.loc[data_ind, "experiment_date"]
well_num = fin_stats_df.loc[data_ind, "well_index"]
time_num = fin_stats_df.loc[data_ind, "time_index"]

# other metadata
chem_i = fin_stats_df.loc[data_ind, "chem_i"]
print(chem_i)
# load points sets
file_stub = f"{date_string}_well{well_num:04}_time{time_num:04}"
fname_c = file_stub + "_smoothed_fin_mesh.obj"
fpath_c = os.path.join(root, "point_cloud_data", "processed_fin_data", fname_c)

mesh = trimesh.load(fpath_c)

DMSO_36


In [13]:
#igl.read_triangle_mesh("/home/nick/projects/flowshape/demo/ABal.obj")
v, f = v, f = mesh.vertices.copy(), mesh.faces.copy()
mp.plot(v, f, shading = {"wireframe":True})

/Users/nick/miniforge3/envs/meshing/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(
/Users/nick/miniforge3/envs/meshing/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(26.087617…

### Run spherical mapping

In [14]:
# normalize the scaling of the mesh
v = fs.normalize(v)

# run the spherical mapping flow and mobius centering
sv = fs.sphere_map(v, f)

# Now we have a spherical mesh
mp.plot(sv, f, shading = {"wireframe":True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5974044…

### Calculate the mean curvature

In [15]:
rho = fs.curvature_function(v, sv, f)

mp.plot(v,f, rho )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0504199…

In [16]:
# Now we have a spherical mesh
mp.plot(sv, f, rho)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5974044…

## SH decomposition

In [18]:
# this utility does the above steps + SH decomposition
# Here, using maximum degree 24
weights, Y_mat, vs = fs.do_mapping(v,f, l_max = 24)

In [19]:
# This is the array of SH weights
np.set_printoptions(threshold = 100)
print(weights)

[ 3.16154044 -0.03579567  0.15363112 ... -0.00534127  0.05360006
  0.05008766]


In [20]:
# Y_mat is a matrix used to convert between weights and mesh function
rho2 = Y_mat.dot(weights)
mp.plot(sv,f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.5974044…

### Mesh reconstruction

In [21]:
rec2 = fs.reconstruct_shape(sv, f, rho2 )
mp.plot(rec2,f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0737382…

### Test with lower frequencies only

In [24]:
weights, Y_mat, vs = fs.do_mapping(v,f, l_max=12)
rec_8 = fs.reconstruct_shape(sv, f, Y_mat.dot(weights) )
mp.plot(rec_8, f, c = rho2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0656042…